<a href="https://colab.research.google.com/github/CharlesCoster/ada-project-terrorism-impact/blob/master/pre_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [16]:
!pip install geopandas

     |████████████████████████████████| 921kB 2.8MB/s 
     |████████████████████████████████| 10.1MB 38.7MB/s 
     |████████████████████████████████| 11.8MB 199kB/s 


TODO


*   GDP across years
*   Population
*   Happyness index
*   Political elections



In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#1. Tourism

In [0]:
import pandas as pd

df_tourism = pd.read_csv('tourism.csv',sep='\t')

In [0]:
# Keep data in national currency and only exporta and import total tourism values
df_tourism = df_tourism[df_tourism.VAR.isin(['EXPORT_TOTAL','IMPORT_TOTAL'])&df_tourism.CUR.isin(['NAT'])]

# Compute increase since last year
df_tourism['Growth'] = df_tourism.groupby(['COUNTRY','VAR']).Value.pct_change()
df_tourism = df_tourism[['COUNTRY','VAR','YEAR','Growth']]
df_tourism.columns=['country_code','tourism_var','year','tourism_growth']
df_tourism.head()

,country_code,tourism_var,year,tourism_growth
0,AUS,EXPORT_TOTAL,2008,NaN
1,AUS,EXPORT_TOTAL,2009,0.034807
2,AUS,EXPORT_TOTAL,2010,-0.002731
3,AUS,EXPORT_TOTAL,2011,-0.021077
4,AUS,EXPORT_TOTAL,2012,-0.021957


In [0]:
df_tourism.set_index(['country_code','year'],inplace=True)
df_tourism = df_tourism.pivot_table(index=df_tourism.index,columns='tourism_var',values='tourism_growth')
df_tourism.reset_index(inplace=True)
df_tourism.head()

tourism_var,index,EXPORT_TOTAL,IMPORT_TOTAL
0,"(AUS, 2009)",0.034807,-0.002004
1,"(AUS, 2010)",-0.002731,0.058791
2,"(AUS, 2011)",-0.021077,0.097287
3,"(AUS, 2012)",-0.021957,0.053171
4,"(AUS, 2013)",0.043211,0.093014


In [0]:
df_tourism.columns=['country_year','growth_export_tourism','growth_import_tourism']
df_tourism['country_code'] = df_tourism.country_year.apply(lambda x:x[0])
df_tourism['year'] = df_tourism.country_year.apply(lambda x:x[1])
df_tourism.drop('country_year',axis=1,inplace=True)
df_tourism.head()

,growth_export_tourism,growth_import_tourism,country_code,year
0,0.034807,-0.002004,AUS,2009
1,-0.002731,0.058791,AUS,2010
2,-0.021077,0.097287,AUS,2011
3,-0.021957,0.053171,AUS,2012
4,0.043211,0.093014,AUS,2013


In [0]:
df_tourism.to_csv('final_tourism.csv')

#2. Terrorism

In [0]:
# We import dblp from google drive
id='1bfZzGpT5jQwYuLHfdgPO_4JSLyPsIVBR'

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('globalterrorismdb_0919dist.xlsx')  
dblp = pd.read_excel('globalterrorismdb_0919dist.xlsx',index_col='eventid')

In [0]:
dblp.head()

,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,...,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
eventid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,...,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
197001000003,1970,1,0,NaN,0,NaT,101,Japan,4,East Asia,Fukouka,Fukouka,33.580412,130.396361,1.0,0,NaN,NaN,1,1,1,-9,NaN,NaN,0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Consulate,217.0,...,NaN,NaN,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [0]:
# Keep only datas with political social economical or social goal, with intention to coerce,... and outsie international humanitarian law
terr_df = dblp[dblp.crit1.isin([1])&dblp.crit2.isin([1])&dblp.crit3.isin([1])]

# Keep variabls of interest
features_col = ['iyear','imonth','iday','country_txt','city','region_txt','attacktype1_txt','attacktype2_txt','attacktype3_txt',
                'targtype1_txt','targtype2_txt','weaptype1_txt','weaptype2_txt','weaptype3_txt','weaptype4_txt','gname',
                'nkill','nkillter']
              
terr_df = terr_df[features_col]

terr_df.to_csv('final_terrorism.csv')

# 1. Terrorism evolution

In [0]:
# Weapon occurence per year
terr_df.groupby('iyear').weaptype1_txt','weaptype2_txt','weaptype3_txt']].value_counts(normalize=True)

AttributeError: ignored

In [0]:
# Attack type occurence per year

In [0]:
# Terrorist groups occurence per year

#4. Correlations

In [0]:
df_terrorism = dblp.groupby(['iyear','country_txt']).size().to_frame().reset_index()
df_new = dblp.groupby(['iyear','country_txt']).nkill.sum().to_frame().reset_index()
df_terrorism.columns=['year','country','nattacks']
df_new.columns=['year','country','nkills']

In [0]:
df_terrorism = df_terrorism.merge(df_new)

In [0]:
country_code = pd.read_csv('country_code.csv',encoding='latin-1')
country_code.columns=['country','country_code']
df_terrorism = df_terrorism.merge(country_code,on='country',how='left')

In [0]:
df_terrorism.drop('country',axis=1,inplace=True)
df_terrorism.head()

,year,nattacks,nkills,country_code
0,1970,21,5.0,ARG
1,1970,1,0.0,AUS
2,1970,1,0.0,BEL
3,1970,1,0.0,BOL
4,1970,6,2.0,BRA


In [0]:
# Get all combinations of year country possible
from itertools import product

year = np.arange(df_terrorism.year.min(),df_terrorism.year.max()+1)
country = list(df_terrorism.country_code.unique())
df_comb = pd.DataFrame(list(product(year, country)), columns=['year', 'country_code'])
df_comb['data'] = 0
df_terrorism = df_comb.merge(df_terrorism,how='left')
df_terrorism.fillna(0,inplace=True)
df_terrorism.drop('data',axis=1,inplace=True)
df_terrorism.head()

,year,country_code,nattacks,nkills
0,1970,ARG,21.0,5.0
1,1970,AUS,1.0,0.0
2,1970,BEL,1.0,0.0
3,1970,BOL,1.0,0.0
4,1970,BRA,6.0,2.0


In [0]:
country_code.head()

,country,country_code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [0]:
df_terrorism[df_terrorism.Country.isna()]['country'].unique()

AttributeError: ignored

In [0]:
df_terrorism.groupby('iyear').nkill.sum().plot()
plt.show()

In [0]:
df_terrorism.plot.scatter(x='iyear',y='nkill')

We will be interested on the most killing attacks, that is attacks with at least 5 kills

In [0]:
kill_df = df_terrorism[df_terrorism.nkill > 5]

In [0]:
year_attacks = kill_df.iyear.unique()

# Politic

In [0]:
cpds = pd.read_excel('http://www.cpds-data.org/images/Update2019/CPDS-1960-2017-Update-2019.xlsx', parse_dates=True)

In [19]:
cpds.head()

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,gov_cent1,gov_left1,gov_party,gov_new,gov_gap,gov_chan,gov_right2,gov_cent2,gov_left2,gov_right3,gov_cent3,gov_left3,gov_sup,gov_type,year_01,country_01,elect,vturn,social1,social2,social3,social4,social5,social6,social7,social8,leftsoc1,leftsoc2,leftsoc3,leftsoc4,...,educexp_gov,educexp_gov_ipol,educexp_public,educexp_public_ipol,educexp_private,educexp_private_ipol,educatt_minimal,educatt_minimal_ipol,educatt_tertiary,educatt_tertiary_ipol,year_11,country_11,fallow_pmp,mpleave_pmp,othfam_c_pmp,childcare_pmp,homehelp_pmp,othfam_k_pmp,servadmi_pmp,training_pmp,jobrot_pmp,incent_pmp,disabled_pmp,jobcrea_pmp,startup_pmp,compen_pmp,earretir_pmp,year_12,country_12,emprot_reg,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960,Australia,1,AUS,36,1,0,0,0,100.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,0.0,63.10,0.0,0.0,63.10,2.0,1960,Australia,NaT,95.5,42.8,7.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1960,Australia,NaN,NaN,NaN,NaN,NaN,10289.707031,6239.689941,871.096008,8.465703,1960,Australia
1,1961,Australia,1,AUS,36,1,0,0,0,100.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,62.80,0.0,0.0,62.80,2.0,1961,Australia,1961-12-09,95.3,47.9,7.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961,Australia,NaN,NaN,NaN,NaN,NaN,10493.554688,6370.598145,890.710999,8.488172,1961,Australia
2,1962,Australia,1,AUS,36,1,0,0,0,100.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,0.0,50.80,0.0,0.0,50.80,2.0,1962,Australia,NaT,95.3,47.9,7.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962,Australia,NaN,NaN,NaN,NaN,NaN,10691.689453,6512.208984,909.627991,8.507805,1962,Australia
3,1963,Australia,1,AUS,36,1,0,0,0,100.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0,0.0,0.0,51.14,0.0,0.0,51.14,2.0,1963,Australia,1963-11-30,95.7,45.5,7.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963,Australia,NaN,NaN,NaN,NaN,NaN,10894.269531,6650.351074,928.943970,8.526904,1963,Australia
4,1964,Australia,1,AUS,36,1,0,0,0,100.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,0.0,59.00,0.0,0.0,59.00,2.0,1964,Australia,NaT,95.7,45.5,7.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964,Australia,NaN,NaN,NaN,NaN,NaN,11115.210938,6795.431152,943.978027,8.492668,1964,Australia


In [0]:
df_politic = cpds[['year','iso']].copy()
df_politic.columns = ['year','country_code']

In [0]:
df_politic['elec_leftsoc'] = cpds['leftsoc1']+cpds['leftsoc2']+cpds['leftsoc3']+cpds['leftsoc4']+cpds['leftsoc5']
df_politic['elec_comm'] = cpds['comm1']+cpds['comm2']+cpds['comm3']+cpds['comm4']
df_politic['elec_conserv'] = cpds['conserv1']+cpds['conserv2']+cpds['conserv3']+cpds['conserv4']+cpds['conserv5']+cpds['conserv6']+cpds['conserv7']+cpds['conserv8']
df_politic['elec_right'] = cpds['right1']+cpds['right2']+cpds['right3']+cpds['right4']+cpds['right5']+cpds['right6']

In [0]:
df_politic['left_soc_inc'] = df_politic.groupby(['country_code']).elec_leftsoc.diff()
df_politic['comm_inc'] = df_politic.groupby(['country_code']).elec_comm.diff()
df_politic['conserv_inc'] = df_politic.groupby(['country_code']).elec_conserv.diff()
df_politic['right_inc'] = df_politic.groupby(['country_code']).elec_right.diff()
df_politic.drop(['elec_leftsoc','elec_comm','elec_conserv','elec_right'],axis=1,inplace=True)
df_politic.head()

,year,country_code,left_soc_inc,comm_inc,conserv_inc,right_inc
0,1960,AUS,NaN,NaN,NaN,NaN
1,1961,AUS,0.0,0.0,-3.6,0.0
2,1962,AUS,0.0,0.0,0.0,0.0
3,1963,AUS,0.0,0.0,3.5,0.0
4,1964,AUS,0.0,0.0,0.0,0.0


In [0]:
df_politic.left_soc_inc.replace(to_replace=np.float64(0), value=None,inplace=True,method='ffill')
df_politic.comm_inc.replace(to_replace=np.float64(0), value=None,inplace=True,method='ffill')
df_politic.conserv_inc.replace(to_replace=np.float64(0), value=None,inplace=True,method='ffill')
df_politic.right_inc.replace(to_replace=np.float64(0), value=None,inplace=True,method='ffill')

In [0]:
df_politic['turnout'] = cpds['vturn']
df_politic.to_csv('final_politic.csv')

In [23]:
df_politic.head()

,year,country_code,elec_leftsoc,elec_comm,elec_conserv,elec_right,turnout
0,1960,AUS,0.0,0.0,37.2,0.0,95.5
1,1961,AUS,0.0,0.0,33.6,0.0,95.3
2,1962,AUS,0.0,0.0,33.6,0.0,95.3
3,1963,AUS,0.0,0.0,37.1,0.0,95.7
4,1964,AUS,0.0,0.0,37.1,0.0,95.7


# Economic openess

In [0]:
df_openess = cpds[['year','iso','openc','realgdpgr']].copy()
df_openess['opengr'] = df_openess.groupby(['iso']).openc.pct_change()
df_openess.drop('openc',axis=1,inplace=True)
df_openess.columns = ['year','country_code','gdpgr','opengr']
df_openess.head()

,year,country_code,gdpgr,opengr
0,1960,AUS,NaN,NaN
1,1961,AUS,0.040831,-0.030252
2,1962,AUS,5.465480,0.009650
3,1963,AUS,6.141373,0.069650
4,1964,AUS,6.200315,-0.007824


In [0]:
df_openess.to_csv('final_economic.csv')

# Happiness Index

In [0]:
happiness2015_df = pd.read_csv('2015.csv')
happiness2015_df.drop(['Region','Standard Error'],axis=1,inplace=True)
happiness2015_df.columns = ['country','happiness_rank','happiness_score','gdp_per_capita','family','life','freedom','generosity','trust','dystopia']
happiness2015_df['year'] = 2015
happiness2015_df.head()

,country,happiness_rank,happiness_score,gdp_per_capita,family,life,freedom,generosity,trust,dystopia,year
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015


In [0]:
happiness2016_df = pd.read_csv('2016.csv')
happiness2016_df.drop(['Region','Lower Confidence Interval','Upper Confidence Interval'],axis=1,inplace=True)
happiness2016_df.columns= ['country','happiness_rank','happiness_score','gdp_per_capita','family','life','freedom','generosity','trust','dystopia']
happiness2016_df['year'] = 2016
happiness2016_df.head() 

,country,happiness_rank,happiness_score,gdp_per_capita,family,life,freedom,generosity,trust,dystopia,year
0,Denmark,1,7.526,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939,2016
1,Switzerland,2,7.509,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463,2016
2,Iceland,3,7.501,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137,2016
3,Norway,4,7.498,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465,2016
4,Finland,5,7.413,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596,2016


In [0]:
happiness2017_df = pd.read_csv('2017.csv')
happiness2017_df.drop(['Whisker.high','Whisker.low'],axis=1,inplace=True)
happiness2017_df.columns= ['country','happiness_rank','happiness_score','gdp_per_capita','family','life','freedom','generosity','trust','dystopia']
happiness2017_df['year'] = 2017
happiness2017_df.head()

,country,happiness_rank,happiness_score,gdp_per_capita,family,life,freedom,generosity,trust,dystopia,year
0,Norway,1,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,2017
1,Denmark,2,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,2017
2,Iceland,3,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,2017
3,Switzerland,4,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,2017
4,Finland,5,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,2017


In [0]:
happiness_df = happiness2015_df.merge(happiness2016_df,how='outer')
happiness_df = happiness_df.merge(happiness2017_df,how='outer')
happiness_df = happiness_df.merge(country_code,on='country',how='left')
happiness_df.drop(['country','happiness_rank'],axis=1,inplace=True)
happiness_df.head()

,happiness_score,gdp_per_capita,family,life,freedom,generosity,trust,dystopia,year,country_code
0,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015,CHE
1,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015,ISL
2,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015,DNK
3,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015,NOR
4,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015,CAN


# Polity IV

In [2]:
polity_iv = pd.read_excel('p4v2018.xls')
polity_iv.head()

,cyear,ccode,scode,country,year,flag,fragment,democ,autoc,polity,polity2,durable,xrreg,xrcomp,xropen,xconst,parreg,parcomp,exrec,exconst,polcomp,prior,emonth,eday,eyear,eprec,interim,bmonth,bday,byear,bprec,post,change,d4,sf,regtrans
0,21800,2,USA,United States,1800,0,NaN,7,3,4,4.0,NaN,3,3,4,7,4,2,8.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1800.0,1.0,4.0,88.0,1.0,NaN,NaN
1,21801,2,USA,United States,1801,0,NaN,7,3,4,4.0,NaN,3,3,4,7,4,2,8.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21802,2,USA,United States,1802,0,NaN,7,3,4,4.0,NaN,3,3,4,7,4,2,8.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21803,2,USA,United States,1803,0,NaN,7,3,4,4.0,NaN,3,3,4,7,4,2,8.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21804,2,USA,United States,1804,0,NaN,7,3,4,4.0,NaN,3,3,4,7,4,2,8.0,7,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Keep variables of interest, polity2 is an polity rank from autocracy (-10) to democracy (+10)
polity_iv = polity_iv[['scode','year','polity2']]
polity_iv.columns = ['country_code','year','polity_rank']
polity_iv.to_csv('final_polity_iv.csv')

In [14]:
fragility = pd.read_excel('SFIv2018.xls')

#Keep variables of interest, sfi is the fragility index in the range 0,25
fragility.rename({'scode':'country_code'},axis=1,inplace=True)
fragility.drop(['country','effect','legit'],axis=1,inplace=True)
fragility.to_csv('final_fragility.csv')
fragility.head()

,country_code,year,region,sfi,seceff,secleg,poleff,polleg,ecoeff,ecoleg,soceff,socleg
0,AFG,1995,2,25,3,3,3,3,4,3,3,3
1,AFG,1996,2,24,3,3,2,3,4,3,3,3
2,AFG,1997,2,24,3,3,2,3,4,3,3,3
3,AFG,1998,2,24,3,3,2,3,4,3,3,3
4,AFG,1999,2,24,3,3,2,3,4,3,3,3
